In [1]:
import pandas as pd
import requests
import re
import time

In [3]:
# African countries - using lowercase names as they appear in URLs
# URL format: https://www.global-climatescope.org/markets/{country}

african_countries = {
    'algeria': 'DZA',
    'angola': 'AGO',
    'benin': 'BEN',
    'botswana': 'BWA',
    'burkina-faso': 'BFA',
    'burundi': 'BDI',
    'cameroon': 'CMR',
    'cape-verde': 'CPV',
    'central-african-republic': 'CAF',
    'chad': 'TCD',
    'comoros': 'COM',
    'congo': 'COG',
    'congo-drc': 'COD',
    'djibouti': 'DJI',
    'egypt': 'EGY',
    'equatorial-guinea': 'GNQ',
    'eritrea': 'ERI',
    'eswatini': 'SWZ',
    'ethiopia': 'ETH',
    'gabon': 'GAB',
    'gambia': 'GMB',
    'ghana': 'GHA',
    'guinea': 'GIN',
    'guinea-bissau': 'GNB',
    'ivory-coast': 'CIV',
    'kenya': 'KEN',
    'lesotho': 'LSO',
    'liberia': 'LBR',
    'libya': 'LBY',
    'madagascar': 'MDG',
    'malawi': 'MWI',
    'mali': 'MLI',
    'mauritania': 'MRT',
    'mauritius': 'MUS',
    'morocco': 'MAR',
    'mozambique': 'MOZ',
    'namibia': 'NAM',
    'niger': 'NER',
    'nigeria': 'NGA',
    'rwanda': 'RWA',
    'sao-tome-and-principe': 'STP',
    'senegal': 'SEN',
    'seychelles': 'SYC',
    'sierra-leone': 'SLE',
    'somalia': 'SOM',
    'south-africa': 'ZAF',
    'south-sudan': 'SSD',
    'sudan': 'SDN',
    'tanzania': 'TZA',
    'togo': 'TGO',
    'tunisia': 'TUN',
    'uganda': 'UGA',
    'zambia': 'ZMB',
    'zimbabwe': 'ZWE'
}

# Storage for results
results = []

print("Starting to scrape Climatescope...")
print(f"Total countries to try: {len(african_countries)}")
print()

Starting to scrape Climatescope...
Total countries to try: 54



In [4]:
# Scrape each country
for country_name, iso_code in african_countries.items():

    url = f"https://www.global-climatescope.org/markets/{country_name}"

    print(f"Trying {country_name.replace('-', ' ').title():<30s} ", end="")

    try:
        response = requests.get(url, timeout=10)
        text = response.text

        # Look for pattern: "$XXX.XX/MWh in 2024"
        # Example: "$162.15/MWh in 2024"
        pattern = r'\$(\d+\.?\d*)/MWh in 2024'
        found = re.search(pattern, text)

        if found:
            price_mwh = float(found.group(1))  # Price in $/MWh
            price_kwh = price_mwh / 1000  # Convert to $/kWh (divide by 1000)

            if price_kwh > 0:
                results.append({
                    'country': country_name.replace('-', ' ').title(),
                    'iso': iso_code,
                    'tariff_usd_per_kwh': price_kwh,
                    'source': 'Climatescope',
                    'source_url': url
                })
                print(f"SUCCESS: ${price_kwh:.3f}/kWh (${price_mwh:.2f}/MWh)")
            else:
                print(f"FAILED: Found $0")
        else:
            print(f"FAILED: No price found")

        time.sleep(0.5)  # Wait half second

    except:
        print(f"ERROR: Could not access")
        time.sleep(0.5)

# Create DataFrame
df = pd.DataFrame(results)

print()
print("="*70)
print("SCRAPING COMPLETE")
print("="*70)
print(f"Successfully scraped: {len(df)} countries")
print()

# Show results
if len(df) > 0:
    df_sorted = df.sort_values('tariff_usd_per_kwh')

    print(f"{'Country':<30s} {'ISO':<5s} {'Tariff ($/kWh)'}")
    print("-"*70)

    for idx, row in df_sorted.iterrows():
        print(f"{row['country']:<30s} {row['iso']:<5s} ${row['tariff_usd_per_kwh']:.3f}")

Trying Algeria                        SUCCESS: $0.031/kWh ($31.00/MWh)
Trying Angola                         SUCCESS: $0.012/kWh ($12.00/MWh)
Trying Benin                          SUCCESS: $0.201/kWh ($200.81/MWh)
Trying Botswana                       SUCCESS: $0.111/kWh ($111.00/MWh)
Trying Burkina Faso                   SUCCESS: $0.212/kWh ($212.00/MWh)
Trying Burundi                        SUCCESS: $0.116/kWh ($116.00/MWh)
Trying Cameroon                       SUCCESS: $0.147/kWh ($147.04/MWh)
Trying Cape Verde                     FAILED: No price found
Trying Central African Republic       FAILED: No price found
Trying Chad                           SUCCESS: $0.162/kWh ($162.15/MWh)
Trying Comoros                        FAILED: No price found
Trying Congo                          FAILED: No price found
Trying Congo Drc                      FAILED: No price found
Trying Djibouti                       FAILED: No price found
Trying Egypt                          SUCCESS: $0.036/kWh ($

In [7]:
# Save
if len(df) > 0:
    df.to_csv('chart3_tariffs_climatescope.csv', index=False)
    print()
    print("SAVED: tariffs_climatescope.csv")
    print("="*70)
else:
    print("ERROR: No data scraped")


SAVED: tariffs_climatescope.csv
